In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pmdarima import auto_arima
from statsmodels.tsa.seasonal import seasonal_decompose
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [22]:
len(train)

150150

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150150 entries, 0 to 150149
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   record_ID        150150 non-null  int64  
 1   week             150150 non-null  object 
 2   store_id         150150 non-null  int64  
 3   sku_id           150150 non-null  int64  
 4   total_price      150149 non-null  float64
 5   base_price       150150 non-null  float64
 6   is_featured_sku  150150 non-null  int64  
 7   is_display_sku   150150 non-null  int64  
 8   units_sold       150150 non-null  int64  
dtypes: float64(2), int64(6), object(1)
memory usage: 10.3+ MB


In [3]:
train['week']=pd.to_datetime(train['week'])
test['week']=pd.to_datetime(test['week'])

In [4]:
train['discount']=train['base_price']-train['total_price']
test['discount']=test['base_price']-test['total_price']

In [5]:
from sklearn.preprocessing import OrdinalEncoder

In [9]:
test['units_sold']=-1

In [11]:
train1=train.append(test)

In [12]:
train1

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,discount,Week_no
0,1,2011-01-17,8091,216418,99.0375,111.8625,0,0,20,12.8250,1.0
1,2,2011-01-17,8091,216419,99.0375,99.0375,0,0,28,0.0000,1.0
2,3,2011-01-17,8091,216425,133.9500,133.9500,0,0,19,0.0000,1.0
3,4,2011-01-17,8091,216233,133.9500,133.9500,0,0,44,0.0000,1.0
4,5,2011-01-17,8091,217390,141.0750,141.0750,0,0,52,0.0000,1.0
...,...,...,...,...,...,...,...,...,...,...,...
13855,232281,2013-01-10,9984,223245,241.5375,241.5375,0,0,-1,0.0000,NaN
13856,232282,2013-01-10,9984,223153,240.8250,240.8250,0,0,-1,0.0000,NaN
13857,232285,2013-01-10,9984,245338,382.6125,401.8500,1,1,-1,19.2375,NaN
13858,232286,2013-01-10,9984,547934,191.6625,191.6625,0,0,-1,0.0000,NaN


In [13]:
enc=OrdinalEncoder()
train1['Week_no']=enc.fit_transform(train1['week'].values.reshape(-1,1))


In [16]:
train1['Week_no']=train1['Week_no']%52

In [36]:
train1['month_no']=(train1['Week_no']/4).astype(int)

In [41]:
train1

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,discount,Week_no,month_no,month_sine
0,1,2011-01-17,8091,216418,99.0375,111.8625,0,0,20,12.8250,1.0,0,0.0
1,2,2011-01-17,8091,216419,99.0375,99.0375,0,0,28,0.0000,1.0,0,0.0
2,3,2011-01-17,8091,216425,133.9500,133.9500,0,0,19,0.0000,1.0,0,0.0
3,4,2011-01-17,8091,216233,133.9500,133.9500,0,0,44,0.0000,1.0,0,0.0
4,5,2011-01-17,8091,217390,141.0750,141.0750,0,0,52,0.0000,1.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13855,232281,2013-01-10,9984,223245,241.5375,241.5375,0,0,-1,0.0000,51.0,12,0.0
13856,232282,2013-01-10,9984,223153,240.8250,240.8250,0,0,-1,0.0000,51.0,12,0.0
13857,232285,2013-01-10,9984,245338,382.6125,401.8500,1,1,-1,19.2375,51.0,12,0.0
13858,232286,2013-01-10,9984,547934,191.6625,191.6625,0,0,-1,0.0000,51.0,12,0.0


In [48]:
train1['month_sine']=np.sin(2*3.1415926535897933*train1['month_no']/12)

In [47]:
np.arcsin(1)

3.141592653589793

In [49]:
train1

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,discount,Week_no,month_no,month_sine
0,1,2011-01-17,8091,216418,99.0375,111.8625,0,0,20,12.8250,1.0,0,0.000000e+00
1,2,2011-01-17,8091,216419,99.0375,99.0375,0,0,28,0.0000,1.0,0,0.000000e+00
2,3,2011-01-17,8091,216425,133.9500,133.9500,0,0,19,0.0000,1.0,0,0.000000e+00
3,4,2011-01-17,8091,216233,133.9500,133.9500,0,0,44,0.0000,1.0,0,0.000000e+00
4,5,2011-01-17,8091,217390,141.0750,141.0750,0,0,52,0.0000,1.0,0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13855,232281,2013-01-10,9984,223245,241.5375,241.5375,0,0,-1,0.0000,51.0,12,-2.449294e-16
13856,232282,2013-01-10,9984,223153,240.8250,240.8250,0,0,-1,0.0000,51.0,12,-2.449294e-16
13857,232285,2013-01-10,9984,245338,382.6125,401.8500,1,1,-1,19.2375,51.0,12,-2.449294e-16
13858,232286,2013-01-10,9984,547934,191.6625,191.6625,0,0,-1,0.0000,51.0,12,-2.449294e-16


In [60]:
stores=train['store_id'].unique()

In [10]:
train[['discount','base_price']].corr()

,discount,base_price
discount,1.000000,0.376067
base_price,0.376067,1.000000


In [64]:
cols=['base_price',
       'is_featured_sku', 'is_display_sku','units_sold','discount','Week_no','month_sine']

In [51]:
train=train1.iloc[:len(train)]
test=train1.iloc[len(train):]

In [54]:
test.drop(['units_sold'],axis=1,inplace=True)

In [52]:
len(train)

150150

In [53]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [55]:
sub=pd.DataFrame({'record_ID':test['record_ID'].values,'units_sold':[0]*len(test)})

In [56]:
test.columns

Index(['record_ID', 'week', 'store_id', 'sku_id', 'total_price', 'base_price',
       'is_featured_sku', 'is_display_sku', 'discount', 'Week_no', 'month_no',
       'month_sine'],
      dtype='object')

In [57]:
sub.head()

,record_ID,units_sold
0,212645,0
1,212646,0
2,212647,0
3,212648,0
4,212649,0


In [58]:
from sklearn.model_selection import GridSearchCV

In [66]:
for store in stores:
    d=train[train['store_id']==store]
    t=test[test['store_id']==store]
    skus=d['sku_id'].unique()
    params={'max_depth':[4,6,8],'learning_rate':[0.001,0.01,0.1]}
    for sku1 in skus:
        k=d[d['sku_id']==sku1]
        v=t[t['sku_id']==sku1]
        mod=LGBMRegressor()
        grid=GridSearchCV(mod,params,verbose=1)
        grid.fit(k[cols].drop(['units_sold'],axis=1),k['units_sold'])
        preds=grid.predict(v[['base_price','is_featured_sku', 'is_display_sku','discount','Week_no','month_sine']])
        rec=v['record_ID'].values
        i=0
        for rec1 in rec:
            sub.loc[sub['record_ID']==rec1,'units_sold']=preds[i]
            i+=1


lBackend with 1 concurrent workers.
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    2.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    1.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    1.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    1.7s

In [69]:
sub['units_sold']=sub['units_sold'].apply(lambda x: x if x>0 else 0)

In [70]:
sub.head()

,record_ID,units_sold
0,212645,20.440103
1,212646,25.607180
2,212647,25.879713
3,212648,27.397195
4,212649,23.492581


In [71]:
sub.to_csv('MySub8.csv',index=False)

In [72]:
grid.best_params_

{'learning_rate': 0.1, 'max_depth': 4}